# Project 2: Political Bias Detection with LSTM

**Workshop Duration:** 45 minutes  
**Dataset:** AllSides News Corpus (3 classes: Left, Center, Right)  
**Expected Accuracy:** 65-80% (this is GOOD for bias detection!)

---

## ⚠️ Important Notes

- This is **harder** than topic classification
- Bias is **subtle** and context-dependent
- 70% accuracy is **good** (not 90%+)
- **Read ETHICS.md** before deploying!

---

## ⚙️ Configuration

In [ ]:
# Import required libraries
import numpy as np
import pandas as pd
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Bidirectional, Dense, Dropout, GlobalMaxPooling1D
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils.class_weight import compute_class_weight
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

print("✓ All libraries imported successfully!")

In [ ]:
# Configuration (NOTE: Different from Project 1!)
MAX_WORDS = 20000        # Larger vocabulary (subtle words matter!)
MAX_LEN = 400            # Longer sequences (need more context)
EMBEDDING_DIM = 300      # Use 300d for better semantics
LSTM_UNITS_1 = 128       # First BiLSTM layer
LSTM_UNITS_2 = 64        # Second BiLSTM layer
DROPOUT_RATE = 0.3       # LSTM dropout
DENSE_DROPOUT = 0.5      # Dense dropout
BATCH_SIZE = 32          # Smaller batch size
EPOCHS = 15              # May need more epochs

# File paths
DATA_PATH = 'data/allsides_news_complete.csv'
GLOVE_PATH = 'embeddings/glove.840B.300d.txt'

print("✓ Configuration set!")
print(f"NOTE: Using larger vocab ({MAX_WORDS}) and longer sequences ({MAX_LEN}) than Project 1")

---

## 📊 Part 1: Data Loading and Exploration

**Goal:** Load AllSides dataset and understand class imbalance.

**IMPORTANT:** Watch for class imbalance (fewer Center articles)!

**Claude Code Prompt:**  
`"Load the AllSides dataset and analyze class imbalance"`

In [ ]:
# TODO: Load AllSides dataset
# Hint: Column names might vary (check with df.columns)
# Hint: Handle missing values
# Hint: Ensure labels are integers (0, 1, 2)

def load_data(filepath):
    """
    Load AllSides bias dataset.
    
    Returns:
        X: List of article texts
        y: numpy array of labels (0=Left, 1=Center, 2=Right)
    """
    # TODO: Implement data loading
    pass

# Load the data
# X, y = load_data(DATA_PATH)
# print(f"✓ Loaded {len(X)} articles")

In [ ]:
# TODO: Explore dataset with focus on imbalance
# Hint: Show count AND percentage for each class
# Hint: Calculate average length per bias
# Hint: Show sample articles from each bias

def explore_data(X, y):
    """
    Explore dataset and identify imbalance.
    """
    # TODO: Implement comprehensive exploration
    pass

# Explore the data
# explore_data(X, y)
# print("\n⚠️ NOTE: If Center is underrepresented, we'll use class weights!")

---

## 🔧 Part 2: Text Preprocessing

**Goal:** Preprocess with larger vocabulary and longer sequences.

**Key Differences from Project 1:**
- Larger vocabulary (20,000 vs 10,000)
- Longer sequences (400 vs 200)
- **Stratified splitting** to maintain balance

**Claude Code Prompt:**  
`"Preprocess for bias detection: vocab=20000, length=400, stratified split"`

In [ ]:
# TODO: Implement preprocessing for bias detection
# Hint: Use MAX_WORDS=20000 (larger vocab for subtle words)
# Hint: Use MAX_LEN=400 (more context needed)
# Hint: Use stratify parameter in train_test_split

def preprocess_text(X, y, test_size=0.15, val_size=0.15):
    """
    Preprocess text with stratified splitting.
    
    Returns:
        X_train, X_val, X_test: Padded sequences
        y_train, y_val, y_test: Label arrays
        tokenizer: Fitted tokenizer
    """
    # TODO: Implement preprocessing
    pass

# Preprocess
# X_train, X_val, X_test, y_train, y_val, y_test, tokenizer = preprocess_text(X, y)
# print(f"✓ Train: {len(X_train)}, Val: {len(X_val)}, Test: {len(X_test)}")
# print(f"✓ Vocabulary size: {len(tokenizer.word_index)}")

---

## ⚖️ Part 3: Class Weights

**Goal:** Compute class weights to handle imbalance.

**Why:** Center articles are fewer, so we give them higher weight during training.

**Claude Code Prompt:**  
`"Calculate class weights to handle the imbalanced dataset"`

In [ ]:
# TODO: Compute class weights
# Hint: Use sklearn's compute_class_weight
# Hint: Convert to dict format for Keras

def compute_class_weights(y_train):
    """
    Compute class weights for imbalanced data.
    
    Returns:
        class_weights: Dictionary {0: weight0, 1: weight1, 2: weight2}
    """
    # TODO: Implement class weight calculation
    pass

# Compute weights
# class_weights = compute_class_weights(y_train)
# print("Class weights:")
# for cls, weight in class_weights.items():
#     print(f"  Class {cls}: {weight:.2f}")

---

## 🎯 Part 4: GloVe Embeddings (300d)

**Goal:** Load GloVe 300d for better semantic understanding.

**NOTE:** This is **strongly recommended** (not optional) for bias detection!

**Claude Code Prompt:**  
`"Load GloVe 300d embeddings and create embedding matrix"`

In [ ]:
# TODO: Load GloVe 300d (WARNING: ~2GB, takes 1-2 minutes)
# Hint: Print progress every 100k words

def load_glove_embeddings(filepath):
    """
    Load GloVe 300d embeddings.
    
    Returns:
        embeddings_index: Dictionary word -> vector
    """
    # TODO: Implement GloVe loading
    pass

# Load embeddings
# print("Loading GloVe 300d (be patient, ~2GB file)...")
# embeddings_index = load_glove_embeddings(GLOVE_PATH)
# print(f"✓ Loaded {len(embeddings_index)} word embeddings")

In [ ]:
# TODO: Create embedding matrix
# Hint: Track coverage (% of vocab found in GloVe)

def create_embedding_matrix(word_index, embeddings_index):
    """
    Create embedding matrix for vocabulary.
    
    Returns:
        embedding_matrix: numpy array (vocab_size, 300)
    """
    # TODO: Implement embedding matrix creation
    pass

# Create matrix
# embedding_matrix = create_embedding_matrix(tokenizer.word_index, embeddings_index)
# print(f"✓ Embedding matrix shape: {embedding_matrix.shape}")

---

## 🏗️ Part 5: Stacked Bidirectional LSTM

**Goal:** Build more complex model than Project 1.

**Architecture:**
1. Embedding (300d, GloVe)
2. Bidirectional LSTM 1 (128 units, return sequences)
3. Bidirectional LSTM 2 (64 units)
4. Dense (64 units, ReLU, dropout 0.5)
5. Output (3 classes, softmax)

**Why Bidirectional?** Context from both directions helps detect bias.

**Claude Code Prompt:**  
`"Build stacked bidirectional LSTM for bias detection"`

In [ ]:
# TODO: Build stacked bidirectional LSTM
# Hint: Use Bidirectional(LSTM(...)) for both LSTM layers
# Hint: First LSTM needs return_sequences=True
# Hint: Use dropout in LSTM layers AND dense layer

def build_model(vocab_size, embedding_matrix=None):
    """
    Build stacked bidirectional LSTM.
    
    Returns:
        model: Compiled Keras model
    """
    # TODO: Implement model building
    pass

# Build model
# vocab_size = len(tokenizer.word_index) + 1
# model = build_model(vocab_size, embedding_matrix)
# model.summary()
# print("\nNOTE: This model has ~2-3M parameters (more complex than Project 1)")

---

## 🚀 Part 6: Training with Class Weights

**Goal:** Train model while handling class imbalance.

**Expectations:**
- Training will be **slower** (bidirectional LSTMs)
- Accuracy will be **lower** (65-80% is good!)
- Watch for **overfitting**

**Claude Code Prompt:**  
`"Train with class weights and early stopping. Monitor for overfitting."`

In [ ]:
# TODO: Train with class weights
# Hint: Pass class_weight to model.fit()
# Hint: Use EarlyStopping with patience=5
# Hint: Use ReduceLROnPlateau and ModelCheckpoint

def train_model(model, X_train, y_train, X_val, y_val, class_weights=None):
    """
    Train bidirectional LSTM.
    
    Returns:
        history: Training history
    """
    # TODO: Implement training
    pass

# Train
# print("Training (this will take longer than Project 1)...")
# history = train_model(model, X_train, y_train, X_val, y_val, class_weights)
# print("\n✓ Training complete!")

---

## 📈 Part 7: Visualization

**Goal:** Plot training curves and watch for overfitting.

**Claude Code Prompt:**  
`"Plot training curves. Is the model overfitting?"`

In [ ]:
# TODO: Plot training history

def plot_training_history(history):
    """
    Plot training curves.
    """
    # TODO: Implement plotting
    pass

# Plot
# plot_training_history(history)

---

## 🎯 Part 8: Comprehensive Evaluation

**Goal:** Evaluate with focus on per-class performance.

**Questions to answer:**
- Is Center class performing poorly?
- Which biases get confused (Left↔Center vs Right↔Center)?
- Is 70% accuracy acceptable?

**Claude Code Prompt:**  
`"Evaluate model and analyze per-class performance. Which bias is hardest?"`

In [ ]:
# TODO: Comprehensive evaluation
# Hint: Show overall AND per-class metrics
# Hint: Generate confusion matrix with labels
# Hint: Discuss which biases are confused

def evaluate_model(model, X_test, y_test):
    """
    Evaluate with focus on per-class metrics.
    """
    # TODO: Implement evaluation
    pass

# Evaluate
# evaluate_model(model, X_test, y_test)

---

## 🧪 Part 9: Real-World Testing

**Goal:** Test on articles from known sources.

**Claude Code Prompt:**  
`"Test on articles from CNN, Fox News, BBC. Does it match known source bias?"`

In [ ]:
# TODO: Implement prediction function

def predict_bias(text, model, tokenizer):
    """
    Predict bias of article.
    
    Returns:
        bias: String ('Left', 'Center', 'Right')
        confidence: Probability
        all_probs: All class probabilities
    """
    # TODO: Implement prediction
    pass

# Test on real articles
bias_names = ['Left', 'Center', 'Right']

# TODO: Add real article texts here
test_articles = {
    "CNN (Left-leaning)": "[Add real article text]",
    "Fox News (Right-leaning)": "[Add real article text]",
    "BBC (Center)": "[Add real article text]",
}

# TODO: Test predictions
# for source, article in test_articles.items():
#     bias, conf, probs = predict_bias(article, model, tokenizer)
#     print(f"Source: {source}")
#     print(f"Predicted: {bias} ({conf:.2%})")
#     print(f"All probabilities: Left={probs[0]:.2%}, Center={probs[1]:.2%}, Right={probs[2]:.2%}\n")

---

## 💭 Part 10: Ethical Discussion

**CRITICAL:** Before deploying, discuss these questions:

### Questions to Consider:

1. **What is bias?**
   - Is "center" objectively definable?
   - Does bias = incorrect?

2. **Model limitations:**
   - 70% accuracy means 30% errors
   - Model sees patterns, not truth
   - Context matters

3. **Potential harms:**
   - Could this be used for censorship?
   - Might it increase polarization?
   - What about false positives?

4. **Responsible use:**
   - Education: YES ✅
   - Content moderation: Careful ⚠️
   - Automated filtering: NO ❌

**Read ETHICS.md for full discussion!**

---

## 🎉 Congratulations!

You've built a bias detector using advanced LSTMs! 🚀

### Key Takeaways:
1. ✅ Bias detection is **harder** than topic classification
2. ✅ 70% accuracy is **good** for this task
3. ✅ Bidirectional LSTMs capture **context**
4. ✅ Class imbalance needs **special handling**
5. ✅ **Ethics matter** - use responsibly!

### Next Steps:
1. Compare with BERT/transformers
2. Add attention mechanism
3. Build explainability tools (LIME/SHAP)
4. Create web interface
5. **Most importantly:** Read ETHICS.md!

### Save Your Model:
```python
model.save('bias_detector_bilstm.h5')
```

### Convert to Python Script:
```bash
jupyter nbconvert --to script starter_notebook.ipynb
```

---

**Remember: Build responsibly! 🌟**